# **1. Data Extraction**

**1.1 Installing Required Packages**

In [ ]:
pip install snscrape

In [ ]:
#import tweepy
from textblob import TextBlob
#from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
#import matplotlib.pyplot as plt
import snscrape.modules.twitter as sntwitter
import string

#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
#import random
#from sklearn.decomposition import PCA
#from sklearn.cluster import KMeans
#from sklearn.metrics import silhouette_score
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from google.colab import drive



#plt.style.use('fivethirtyeight')

**1.2 extracting data using twitter Search Scraper**

In [ ]:

query="#RussiaUkraineWar"
tweets=[]
limit=25000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
  #print(vars(tweets))
  if len(tweets)==limit:
    break
  else:
    tweets.append([tweet.date,tweet.username,tweet.content])

df=pd.DataFrame(tweets,columns=['Date','User','Tweet'])
df


,Date,User,Tweet
0,2022-06-19 14:39:17+00:00,SuganthanRamak1,"'For God's sake, it's their choice' - #Putin o..."
1,2022-06-19 14:38:45+00:00,WilsonShilo,The EU warns that #Russian actions in #RussiaU...
2,2022-06-19 14:37:50+00:00,kiselliuda,#Ukraine #RussiaUkraineWar #war #culture #thef...
3,2022-06-19 14:37:19+00:00,NasticInc,🎥A Russian Plane Crashes near Chernihiv!!!!!!!...
4,2022-06-19 14:37:01+00:00,NasticInc,🎥 Russian Military Fleeing an area near Kherso...
...,...,...,...
24995,2022-05-27 00:04:53+00:00,RusskieUkraine,Insane footage from #Ukraine US and UK foreign...
24996,2022-05-27 00:03:51+00:00,Globallpolitics,Obesity is the second leading cause of death i...
24997,2022-05-27 00:01:02+00:00,Globallpolitics,List of 6 anti aging-food. \n#USA #China #Food...
24998,2022-05-27 00:00:41+00:00,Hashmat_M,The #West is supplying #Ukraine with weapons i...


In [ ]:
len(df)

25000

## **2. Data Cleaning**

In [ ]:
# Lower Casing
df['Tweet'] = df.Tweet.str.lower()

#create a function to clean the tweets
def cleantxt(text):
  text=re.sub(r'@[A-Za-z0-9]+', '',text) # It removes @ mentions from the tweets
  text=re.sub(r'#','',text) # It removes # hashtags from the tweets
  text=re.sub(r'RT[\s]+','',text) #It removes re-tweets
  text=re.sub(r'https?:\/\/\S+','',text) # It removes the hyperlink
  text=re.sub(r'[^\x00-\x7F]','',text) # It removes the emojis
  return text

#cleaing the text
df['Tweet']=df['Tweet'].apply(cleantxt)

# Removing Punctuations
def remove_punctuations(text):
  return text.translate(str.maketrans('','', string.punctuation))

df['Tweet'] = df.Tweet.apply(lambda x:remove_punctuations(x))

# Removing extra spaces between words if any
df['Tweet'] = df['Tweet'].apply(lambda x: " ".join(x.split()))

# Removing tweets with non-english words
nltk.download('words')
nltk.download('omw-1.4')
words = set(nltk.corpus.words.words())
to_drop = []

for i in df.index:
  for w in nltk.wordpunct_tokenize(df.loc[i, 'Tweet']):
    if w not in words or w.isalpha()==False:
      to_drop.append(i)
      break
    break

df.drop(to_drop,inplace = True)

# Lemmatization
nltk.download('wordnet')
le = WordNetLemmatizer()
def lemmatize_text(text):
  return ' '.join([le.lemmatize(word,pos='v') for word in text.split()])

df['Tweet'] = df['Tweet'].apply(lambda x: lemmatize_text(x))


# saving the dataframe
drive.mount('/content/drive')
path = '/content/drive/My Drive/SentimentAnalysis/dataset.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)

#df.to_csv(r'C:\Users\anjali\SentimentAnalysis-RussiaUkraineWar\data\marchdataset.csv', index=False)
#show the clean text
print('Length of the Dataframe after removing non-english tweets: ', df.shape[0])
df

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Length of the Dataframe after removing non-english tweets:  11250


,Date,User,Tweet,Subjectivity,Polarity,Analysis,Label
0,2022-06-19 14:39:17+00:00,SuganthanRamak1,for gods sake its their choice putin on ukrain...,0.000000,0.000000,Neutral,2
1,2022-06-19 14:38:45+00:00,WilsonShilo,the eu warn that russian action in russiaukrai...,0.050000,0.050000,Positive,1
3,2022-06-19 14:37:19+00:00,NasticInc,a russian plane crash near chernihiv ukraine r...,0.133333,0.033333,Positive,1
6,2022-06-19 14:35:51+00:00,NasticInc,terror attack catch on cctv footage standwithu...,0.000000,0.000000,Neutral,2
7,2022-06-19 14:35:23+00:00,NasticInc,a russian ship burn at port of berdyansk ukrai...,0.000000,0.000000,Neutral,2
...,...,...,...,...,...,...,...
24994,2022-05-27 00:11:50+00:00,ichbin_ironhill,russia doesnt kill the 5000 ukrainian pows ukr...,0.333333,0.200000,Positive,1
24995,2022-05-27 00:04:53+00:00,RusskieUkraine,insane footage from ukraine us and uk foreign ...,0.545000,-0.265000,Negative,0
24996,2022-05-27 00:03:51+00:00,Globallpolitics,obesity be the second lead cause of death in t...,0.000000,0.000000,Neutral,2
24997,2022-05-27 00:01:02+00:00,Globallpolitics,list of 6 anti agingfood usa china food antiag...,0.000000,0.000000,Neutral,2


In [ ]:
len(df)

11251

## **Splitting Data into training and testing dataset**

In [ ]:
# training_data, testing_data = train_test_split(df, test_size=0.3, random_state=25)
# #Saving training dataset
# path = '/content/drive/My Drive/SentimentAnalysis/training_dataset.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   training_data.to_csv(f)
# #Saving testing dataset
# path = '/content/drive/My Drive/SentimentAnalysis/testing_dataset.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   testing_data.to_csv(f)

In [ ]:
#len(training_data)


7875

In [ ]:
#len(testing_data)

3376

In [ ]:
df

NameError: ignored

# **3. Data Insights**

**3.1 Subjectivity & Polarity**

In [ ]:
#create a function to get a Subjectivity
def getsubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#create a function to get a Polarity
def getpolarity(text):
  return TextBlob(text).sentiment.polarity

#create two new column
df['Subjectivity']=df['Tweet'].apply(getsubjectivity)
df['Polarity']=df['Tweet'].apply(getpolarity)

#Show new dataframe with new column
df

,Date,User,Tweet,Subjectivity,Polarity
19646,2022-05-31 12:07:53+00:00,Emolclause,break polands president andrzejduda appear on ...,0.000000,0.000000
6017,2022-06-13 22:07:01+00:00,Boba12340769066,dont forget the defenders of mariupol russiauk...,0.000000,0.000000
10530,2022-06-09 04:38:52+00:00,TV9Bharatvarsh,world explosive top 25 warcoverageontv9 tv9bha...,0.500000,0.500000
5962,2022-06-13 23:40:44+00:00,hizbie_,delivery come to the ukrainian trench russiauk...,0.000000,0.000000
16251,2022-06-03 10:32:25+00:00,gmbutts,i dont subscribe to the putin chessmaster theo...,0.700000,0.266667
...,...,...,...,...,...
19096,2022-05-31 17:31:27+00:00,Therealgsns,alert there be no more bread russia to cutoff ...,0.450000,-0.075000
6494,2022-06-13 13:53:48+00:00,NasticInc,a russian tank in use by the ukrainian militar...,0.033333,-0.033333
23163,2022-05-28 15:51:09+00:00,WilhelmOrmstein,its not macron in france its not draghi in ita...,0.000000,0.000000
15022,2022-06-04 11:18:25+00:00,FarayMarie,s article demonstrate their paternalistic and ...,0.516667,0.266667


**3.2 Analysis of tweets**

In [ ]:
#create a function to compute the negative,Neutral and possitive analysis
def getanalysis(score):
  if score<0:
    return 'Negative'
  elif score==0:
    return 'Neutral'
  else:
    return 'Positive'

# create a function to Put the Labels in dataframe
def getlabel(score):
  if score<0:
    #negative
    return 0
  elif score==0:
    #neutral
    return 2
  else:
    #positive
    return 1

df['Analysis']=df['Polarity'].apply(getanalysis)
df['Label']=df['Polarity'].apply(getlabel)

#show the Dataframe
df

,Date,User,Tweet,Subjectivity,Polarity,Analysis,Label
19646,2022-05-31 12:07:53+00:00,Emolclause,break polands president andrzejduda appear on ...,0.000000,0.000000,Neutral,2
6017,2022-06-13 22:07:01+00:00,Boba12340769066,dont forget the defenders of mariupol russiauk...,0.000000,0.000000,Neutral,2
10530,2022-06-09 04:38:52+00:00,TV9Bharatvarsh,world explosive top 25 warcoverageontv9 tv9bha...,0.500000,0.500000,Positive,1
5962,2022-06-13 23:40:44+00:00,hizbie_,delivery come to the ukrainian trench russiauk...,0.000000,0.000000,Neutral,2
16251,2022-06-03 10:32:25+00:00,gmbutts,i dont subscribe to the putin chessmaster theo...,0.700000,0.266667,Positive,1
...,...,...,...,...,...,...,...
19096,2022-05-31 17:31:27+00:00,Therealgsns,alert there be no more bread russia to cutoff ...,0.450000,-0.075000,Negative,0
6494,2022-06-13 13:53:48+00:00,NasticInc,a russian tank in use by the ukrainian militar...,0.033333,-0.033333,Negative,0
23163,2022-05-28 15:51:09+00:00,WilhelmOrmstein,its not macron in france its not draghi in ita...,0.000000,0.000000,Neutral,2
15022,2022-06-04 11:18:25+00:00,FarayMarie,s article demonstrate their paternalistic and ...,0.516667,0.266667,Positive,1


In [ ]:
# Saving Training dataset into csv file
path = '/content/drive/My Drive/SentimentAnalysis/final_dataset.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
 df.to_csv(f)

Subjectivity Quantifies the amount of personal opinion and factual information contained in the text. the higher the subjectivity means th text contains personal opinion rather than factual information.

Polarity quantifies this sentiment is often as possitive,neutal or negative from the sign of polarity score.

**3.3 Possitive Tweets**

In [ ]:
#print all of the positive tweets
j=1
sortedDF= df.sort_values(by=['Polarity'])
sortedDF.reset_index(drop=True,inplace=True)
for i in range(0, sortedDF.shape[0]):
  if(sortedDF['Analysis'][i] == 'Positive'):
    print(str(j) + ')' + sortedDF['Tweet'][i])
    print()
    j=j+1

Streaming output truncated to the last 5000 lines.
1240)the longer the ukrainerussian war go on the more it will solidify and galvanize other countries on russian border to join nato and raise stronger armies which be already happen for russia this be already a loss russia ukraine ukrainewar russiaukrainewar

1241)stark depiction of just how much territory russia have be able to occupy red regions below in 110 days of russiaukrainewar

1242)why do it seem like everyone in the last 3 weeks have totally stappoed talk about ukraine be people just ok with the russiaukrainewar and all the russian war crimes

1243)phrase putin war be not correct not only putin be at war against ukraine but about 150 000 russian troop be at war against ukrainians too and many russian diasporas support this war and about 70 russians agree with putin decision to kill ukrainians russiaukrainewar

1244)the russian btr80 withstand a hit from a german matador grenade launcher and continue to move later he be hit by

**3.4 Negative Tweets**

In [ ]:
#print the negative tweets
j=1
sortedDF= df.sort_values(by=['Polarity'], ascending='false')
sortedDF.reset_index(drop=True,inplace=True)
for i in range(0, sortedDF.shape[0]):
  if(sortedDF['Analysis'][i] == 'Negative'):
    print(str(j) + ')' + sortedDF['Tweet'][i])
    print()
    j=j+1

1)this be terrible russiaukrainewar

2)worst in the business do not use cartrack petroldieselprice malema russiaukrainewar

3)i hope putin and his his minions receive the karma they deserve for all these horrible war crimes ukraineunderattak russiaukrainewar putinwarcriminal istandwithukraine

4)what russia be do at the moment be inhuman beyond comprehensionit be time to stop evil putinthe west must wake uprussiaukrainewar

5)why would they war be awful and fight by pawn instead of by kingsrussiaukrainewar

6)today be the 97th day of the cruel war meanwhile continue to do business in russia and finance this war the kill rap and torture stop your business in russia stop fund this war stoprussia russiaukrainewar standwithukraine

7)this be shock evil at its worst putinwarcriminal russiaukrainewar

8)terrible la batalla por severodonetsk es una verdadera locuracalleporcalle casapor casa ukrainerussiawar ukrainewar russiaukrainewar via

9)terrible footage of the ukranian mlrs strike on the

**3.5 Neutral Tweets**

In [ ]:
#print the neutral tweets
j=1
sortedDF= df.sort_values(by=['Polarity'], ascending='false')
sortedDF.reset_index(drop=True,inplace=True)
for i in range(0, sortedDF.shape[0]):
  if(sortedDF['Analysis'][i] == 'Neutral'):
    print(str(j) + ')' + sortedDF['Tweet'][i])
    print()
    j=j+1

Streaming output truncated to the last 5000 lines.
2602)us mishra speak internationalnews russiaukrainewar warcoverageontv9

2603)current situation to buy petrol in pakistan russiaukrainewar fuelprice

2604)the jurys still out on the identity of marcus coppockif hes still in the 1776restorationmovement campor not well get to the bottom of this hopefully by tomorrow pfizerdocuments vaccinesideeffects vaccinedeaths ai russiaukrainewar bitcoin airdrops peoplesconvoy

2605)may 25 1999 serbia nato bomb balkans war russiaukrainewar russiaukraineconflict usa

2606)due to russiaukrainewar ukraine be not able to export wheat so sharp wheat price shoot up will boost itc revenue for sure we be bullish on itc komalsecurities stockmarket nse invest longterminvestment portfolio multibagger

2607)spend time go over account of ukr citizens fight in front line russiaukrainewar vs public statements by ukr political leaders ukr be fight on behalf of europe or nato or while such word may inspire internati

## ***4. Data Visualisation***

In [ ]:
#Get the Percentage of positive tweets
postweets= df[df.Analysis == 'Positive']
postweets = postweets['Tweet']

round((postweets.shape[0] / df.shape[0]) *100,1)

33.2

In [ ]:
#Get the Percentage of negative tweets
negtweets= df[df.Analysis == 'Negative']
negtweets = negtweets['Tweet']

round((negtweets.shape[0] / df.shape[0]) *100,1)

21.4

In [ ]:
#Get the Percentage of Neutral tweets
neutweets= df[df.Analysis == 'Neutral']
neutweets = neutweets['Tweet']

round((neutweets.shape[0] / df.shape[0]) *100,1)

45.3